In [6]:

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
    
import numpy as np

import QUANTAXIS as QA

import Ind_Model_Base

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from tools.Test_Tools import FactorTest
import tools.Sample_Tools as smpl


import pandas as pd



%load_ext autoreload
%autoreload 1
%aimport tools.Test_Tools,tools.Sample_Tools


class Ind_Test(Ind_Model_Base.Ind_Model):
    def __init__(self,data, frequence=QA.FREQUENCE.DAY):
        super().__init__(data, 'TEST', frequence)

    
    def on_set_params_default(self):
        return {}
    
        
    def on_indicator_structuring(self, data):
        return self.excute_for_multicode(data, self.kernel, **self.pramas)

    
    def on_desition_structuring(self, data, ind_data):
        return None
        
    def kernel(self,dataframe):
        CLOSE = dataframe.close
        ret=(CLOSE-CLOSE.shift(1))/CLOSE.shift(1)
        res = alpha1(CLOSE, ret)
        
        return pd.DataFrame({'main': res}, index=dataframe.index)
    
    
    def plot(self,figsize=(1120/72,420/72)):
        pass
    



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

# 移动求和
def ts_sum(df, window):
    return df.rolling(window).sum()
# 移动平均


def sma(df, window):
    return df.rolling(window).mean()

# 移动标准差
def stddev(df, window):
    return df.rolling(window).std()

# 移动相关系数
def correlation(x, y, window):
    return x.rolling(window).corr(y)

# 移动协方差
def covariance(x, y, window):
    return x.rolling(window).cov(y)

# 在过去d天的时序排名
def rolling_rank(na):
    return rankdata(na)[-1]


def ts_rank(df, window):
    return df.rolling(window).apply(rolling_rank)

# 过去d天的时序乘积
def rolling_prod(na):
    return np.prod(na)


def product(df, window):
    return df.rolling(window).apply(rolling_prod)

# 过去d天最小值
def ts_min(df, window):
    return df.rolling(window).min()

# 过去d天最大值
def ts_max(df, window):
    return df.rolling(window).max()

# 当天取值减去d天前的值
def delta(df, period):
    return df.diff(period)

# d天前的值，滞后值
def delay(df, period):
    return df.shift(period)
# 截面数据排序，输出boolean值


def rank(df):
    return df.rank(pct=True, axis=0)

# 缩放时间序列，使其和为1
def scale(df, k=1):
    return df.mul(k).div(np.abs(df).sum())
# 过去d天最大值的位置


def ts_argmax(df, window):
    return df.rolling(window).apply(np.argmax) + 1

# 过去d天最小值的位置
def ts_argmin(df, window):
    return df.rolling(window).apply(np.argmin) + 1

# 线性衰减的移动平均加权
def decay_linear(df, period):
    if df.isnull().values.any():
        df.fillna(method='ffill', inplace=True)
        df.fillna(method='bfill', inplace=True)
        df.fillna(value=0, inplace=True)
    na_lwma = np.zeros_like(df)  # 生成与df大小相同的零数组
    na_lwma[:period, :] = df.iloc[:period, :]  # 赋前period项的值
    na_series = df.as_matrix()
    # 计算加权系数
    divisor = period * (period + 1) / 2
    y = (np.arange(period) + 1) * 1.0 / divisor
    # 从第period项开始计算数值
    for row in range(period - 1, df.shape[0]):
        x = na_series[row - period + 1: row + 1, :]
        na_lwma[row, :] = (np.dot(x.T, y))
    return pd.DataFrame(na_lwma, index=df.index, columns=df.columns)

    

In [8]:
data = smpl.get_data(['000001','000004'],end='2021-07-02', gap=100)

def alpha1(close, returns):
    x = close
    
    x[returns < 0] = stddev(returns, 20)
    alpha = rank(ts_argmax(x ** 2, 5))-0.5

    return alpha.fillna(value=0)

ttt = Ind_Test(data)
ttt.fit()
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
print(ttt.ind_df)

                       main
date       code            
2021-02-02 000001  0.000000
           000004  0.000000
2021-02-03 000001  0.000000
           000004  0.000000
2021-02-04 000001  0.000000
...                     ...
2021-06-30 000004  0.151316
2021-07-01 000001 -0.426667
           000004 -0.039474
2021-07-02 000001  0.100000
           000004 -0.236842

[198 rows x 1 columns]


In [68]:
# import Sample_Tools as smpl
try:
    import talib
except:
    print('PLEASE install TALIB to call these methods')
    pass

def AROONOSC(DataFrame, N=3):
    res = talib.EMA(DataFrame.close.values, N)
    print('----------')
    print(res)
    return pd.DataFrame({'ema': res}, index=DataFrame.index)

data = smpl.get_data(['000001','000004'],end='2021-07-02', gap=10)
ff = data.data
print(ff)
# print(talib.MA(data.data.open, timeperiod=3, matype=0))

# data.add_func(AROONOSC)
ff[['open','high']]=ff[['open','high']].agg('log')
print(ff)

                    open   high    low  close     volume        amount
date       code                                                       
2021-06-18 000001  23.13  23.28  22.39  22.65   694099.0  1.574438e+09
           000004  20.20  20.68  19.55  19.78   106875.0  2.143071e+08
2021-06-21 000001  22.60  22.67  22.01  22.16   718272.0  1.600120e+09
           000004  19.50  21.40  19.18  20.77   113211.0  2.304986e+08
2021-06-22 000001  22.38  23.09  22.25  22.97   757751.0  1.720058e+09
           000004  20.30  20.94  20.01  20.18    78693.0  1.597537e+08
2021-06-23 000001  22.85  23.41  22.72  23.14   447907.0  1.035836e+09
           000004  20.26  20.40  19.66  19.80    65989.0  1.313061e+08
2021-06-24 000001  23.03  23.12  22.68  23.08   390283.0  8.945021e+08
           000004  20.10  20.20  18.82  18.86    79985.0  1.556011e+08
2021-06-25 000001  23.08  23.60  23.06  23.36   559972.0  1.310287e+09
           000004  18.86  19.10  17.90  18.11    85398.0  1.559673e+08
2021-0

In [15]:
%%time
test = FactorTest(Ind_Test, sample='沪深300',freq="m",end='2016-01-29',gap=500,only_main=True,neutralize={'enable':False,'static_mv':False})
# test = FactorTest(Ind_Test, sample='沪深300',freq="m",end='2020-06-29',gap=2500,only_main=True,neutralize={'enable':False,'static_mv':False})
test.process()
# test.plot()

Wall time: 22.2 s


In [2]:
%%time
tt = smpl.get_sample_by_zs('沪深300', end='2016-01-29',gap=2500,only_main=True)

Wall time: 5.51 s


In [3]:
%%time
# print(tt.data)
print(tt.to_qfq().data)

                        open       high        low      close     volume  \
date       code                                                            
2005-10-20 000002   1.110340   1.121846   1.055686   1.107463  1885512.0   
           000063   4.972785   5.080005   4.921024   5.076308    16680.0   
           000066   2.371709   2.383125   2.308920   2.337460    17423.0   
           000069   1.263005   1.307582   1.258052   1.275387    37441.0   
           000100   0.659399   0.665864   0.646469   0.659399    49116.0   
...                      ...        ...        ...        ...        ...   
2016-01-29 603799   8.042174   8.449717   7.721574   8.384510    57434.0   
           603806  13.478810  14.285522  13.478810  14.260312    30039.0   
           603899  15.012016  15.484092  14.964808  15.266937    16346.0   
           603939  13.695875  14.300419  13.155530  14.166670    17110.0   
           603993   2.852105   3.075800   2.833464   2.982594   548002.0   

           